In [1]:
import pandas as pd
from acti_motus import Features, Activities
from pathlib import Path


def rotate_by_90_degrees_over_x(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    y_new = -df['acc_z']
    z_new = df['acc_y']

    df['acc_y'] = y_new
    df['acc_z'] = z_new

    return df


files = Path('data/thigh').glob('*.parquet')

features = Features()
activities = Activities('Sens', orientation=False)

results = []

for file in files:
    gt = file.parent.parent / 'ground_truth' / file.name

    if not gt.exists():
        print(f'Ground truth file does not exist for {file.name}, skipping.')
        continue

    ground_truth = pd.read_parquet(file.parent.parent / 'ground_truth' / file.name)
    df = pd.read_parquet(file)
    df = rotate_by_90_degrees_over_x(df)

    extracted_features = features.extract(df)
    activity, references = activities.detect(extracted_features)

    df = ground_truth.join(activity, how='left')
    results.append(df)

results = pd.concat(results)
results = results.loc[~((results['id'] == 'P17') & (results['ground_truth'] == 'running_fast'))]
results.to_parquet('processed_data.parquet', index=True)

Ground truth file does not exist for P16.parquet, skipping.


Ground truth file does not exist for P13.parquet, skipping.
Ground truth file does not exist for P35.parquet, skipping.
